In [1]:
import os
import datajoint as dj
dj.config['database.host'] = os.environ['DJ_HOST']
dj.config['database.user'] = os.environ['DJ_USERNAME']
dj.config['database.password'] = os.environ['DJ_PASSWORD']
dj.config['enable_python_native_blobs'] = True
dj.config['display.limit'] = 200
        
name = 'mvi'
os.environ["DJ_SCHEMA_NAME"] = f"metrics_{name}"
dj.config["nnfabrik.schema_name"] = os.environ["DJ_SCHEMA_NAME"]

In [2]:
import re
import torch
import numpy as np
import pickle 
import json
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 10)
import matplotlib as mpl
mpl.rcParams["figure.facecolor"] = 'w'
mpl.rcParams["axes.facecolor"] = 'w'
mpl.rcParams["savefig.facecolor"] = 'w'
mpl.rcParams["figure.dpi"] = 100
mpl.rcParams["figure.figsize"] = (3, 3)
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr, pearsonr

from nnsysident.training.trainers import standard_trainer
from nnsysident.models.models import SE2DFullGaussian2d_Poisson, SE2DFullGaussian2d_ZIG, SE2DFullGaussian2d_ZIL
from nnsysident.models.ensemble_models import Ensemble
from nnsysident.utility.data_helpers import extract_data_key
from nnsysident.datasets.mouse_loaders import static_loaders

from neuralpredictors.measures.zero_inflated_losses import ZIGLoss, ZILLoss
from neuralpredictors.measures import corr

from dataport.bcm.static import PreprocessedMouseData

random_seed = 27121992
device = 'cuda'

Connecting konstantin@134.76.19.44:3306


---

In [3]:
# Load model architecture configurations
with open('group233_model_configs.pkl', 'rb') as handle:
    zhiwei_configs = pickle.load(handle)
    
modulator_kwargs = zhiwei_configs["mod_key"]
shifter_kwargs = zhiwei_configs["shift_key"]
modulator_kwargs["bias"] = False
shifter_kwargs["bias"] = False

In [4]:
datasets =  [{'animal_id': 26614,
              'session': 1,
              'scan_idx': 16,
              'scan_purpose': 'imagenet'},
             {'animal_id': 26614,
              'session': 2,
              'scan_idx': 17,
              'scan_purpose': 'dei_control_pair'},
             {'animal_id': 26726,
              'session': 6,
              'scan_idx': 11,
              'scan_purpose': 'imagenet'},
             {'animal_id': 26726,
              'session': 7,
              'scan_idx': 13,
              'scan_purpose': 'dei_control_pair'},
             {'animal_id': 26942,
              'session': 1,
              'scan_idx': 11,
              'scan_purpose': 'imagenet'},
             {'animal_id': 26942,
              'session': 2,
              'scan_idx': 8,
              'scan_purpose': 'dei_control_pair'},
             {'animal_id': 27468,
              'session': 3,
              'scan_idx': 12,
              'scan_purpose': 'imagenet'},
             {'animal_id': 27468,
              'session': 4,
              'scan_idx': 7,
              'scan_purpose': 'dei_control_pair'}]

## Imagenet Data

In [5]:
imagenet_key = datasets[0]
assert imagenet_key["scan_purpose"] == "imagenet"
paths = ["./data/static{}-{}-{}-GrayImageNet-7bed7f7379d99271be5d144e5e59a8e7.zip".format(imagenet_key["animal_id"], imagenet_key["session"], imagenet_key["scan_idx"])]
img_data_key = extract_data_key(paths[0])

dataset_config = {'paths': paths, 
                  'batch_size': 64, 
                  'seed': random_seed,
                  'loader_outputs': ["images", "responses", "pupil_center", "behavior"],
                  # 'loader_outputs': ["images", "responses"],
                  'normalize': True,
                  'exclude': ["images"],
                  'subtract_behavior_mean': True
                  }
    
img_dataloaders = static_loaders(**dataset_config)
img_dataset = img_dataloaders["test"][img_data_key].dataset

---

## Models

### ZIG

In [ ]:
# for random_seed in np.arange(5):
#     loc = np.exp(-10)

#     zig_model_config = {
#         "layers": 4,
#         "hidden_channels": 64,
#         "feature_reg_weight": 0.78,
#         "init_mu_range": 0.55,
#         "init_sigma": 0.4,
#         'grid_mean_predictor': {'type': 'cortex',
#                                   'input_dimensions': 2,
#                                   'hidden_layers': 0,
#                                   'hidden_features': 0,
#                                   'final_tanh': False},
#         'zero_thresholds': {img_data_key: loc},

#         "input_kern": 15,
#         "gamma_input": 1,
#         "hidden_kern": 13,
#         "depth_separable": True,
#         "k_image_dependent": True,
#         "modulator_kwargs": modulator_kwargs,
#         "shifter_kwargs": shifter_kwargs,
#     }


#     zig_model = SE2DFullGaussian2d_ZIG().build_model(img_dataloaders, random_seed, **zig_model_config)
#     zig_model.to(device);

#     score, output, state_dict = standard_trainer(zig_model,
#                                                img_dataloaders,
#                                                random_seed, 
#                                                loss_function="ZIGLoss", 
#                                                stop_function="get_loss", 
#                                                track_training=False, 
#                                                maximize=False)
#     zig_model.eval();
#     torch.save(state_dict, "ZIG_statedict" + img_data_key + f"-seed{random_seed}" + ".inshallah")

In [17]:
loc = np.exp(-10)

zig_model_config = {
    "layers": 4,
    "hidden_channels": 64,
    "feature_reg_weight": 0.78,
    "init_mu_range": 0.55,
    "init_sigma": 0.4,
    'grid_mean_predictor': {'type': 'cortex',
                              'input_dimensions': 2,
                              'hidden_layers': 0,
                              'hidden_features': 0,
                              'final_tanh': False},
    'zero_thresholds': {img_data_key: loc},

    "input_kern": 15,
    "gamma_input": 1,
    "hidden_kern": 13,
    "depth_separable": True,
    "k_image_dependent": True,
    "modulator_kwargs": modulator_kwargs,
    "shifter_kwargs": shifter_kwargs,
}


zig_model = SE2DFullGaussian2d_ZIG().build_model(img_dataloaders, random_seed, **zig_model_config)
zig_model.to(device);

In [ ]:
score, output, state_dict = standard_trainer(zig_model,
                                           img_dataloaders,
                                           random_seed, 
                                           loss_function="ZIGLoss", 
                                           stop_function="get_loss", 
                                           track_training=True, 
                                           maximize=False)
zig_model.eval();
# torch.save(state_dict, "ZIG_statedict" + img_data_key + ".inshallah")


# zig_model.load_state_dict(torch.load("ZIG_statedict" + img_data_key + ".inshallah"))
# zig_model.eval();

val_correlation: 0.29963976

zig_val_loss: -16377972.0

zig_train_loss: -150074690.0

### Poisson

In [ ]:
# for random_seed in np.arange(5):

#     poisson_model_config = {
#         "layers": 4,
#         "hidden_channels": 64,
#         "gamma_readout": 0.78,
#         "init_mu_range": 0.55,
#         "init_sigma": 0.4,
#         'grid_mean_predictor': {'type': 'cortex',
#                                   'input_dimensions': 2,
#                                   'hidden_layers': 0,
#                                   'hidden_features': 0,
#                                   'final_tanh': False},
#         "input_kern": 15,
#         "gamma_input": 1,
#         "hidden_kern": 13,
#         "depth_separable": True,
#         "modulator_kwargs": modulator_kwargs,
#         "shifter_kwargs": shifter_kwargs,  
#     }


#     poisson_model = SE2DFullGaussian2d_Poisson().build_model(img_dataloaders, random_seed, **poisson_model_config)
#     poisson_model.to(device);

#     score, output, state_dict = standard_trainer(poisson_model,
#                                                  img_dataloaders,
#                                                  random_seed,
#                                                  loss_function="PoissonLoss",
#                                                  track_training=False, )
#     poisson_model.eval();
#     torch.save(state_dict, "Poisson_statedict" + img_data_key + f"-seed{random_seed} + ".inshallah"")

In [19]:
poisson_model_config = {
    "layers": 4,
    "hidden_channels": 64,
    "gamma_readout": 0.78,
    "init_mu_range": 0.55,
    "init_sigma": 0.4,
    'grid_mean_predictor': {'type': 'cortex',
                              'input_dimensions': 2,
                              'hidden_layers': 0,
                              'hidden_features': 0,
                              'final_tanh': False},
    "input_kern": 15,
    "gamma_input": 1,
    "hidden_kern": 13,
    "depth_separable": True,
    "modulator_kwargs": modulator_kwargs,
    "shifter_kwargs": shifter_kwargs,  
}

poisson_model = SE2DFullGaussian2d_Poisson().build_model(img_dataloaders, random_seed, **poisson_model_config)
poisson_model.to(device);

In [ ]:
score, output, state_dict = standard_trainer(poisson_model,
                                             img_dataloaders,
                                             random_seed,
                                             loss_function="PoissonLoss",
                                             track_training=True, )
poisson_model.eval();
# torch.save(state_dict, "Poisson_statedict" + img_data_key + ".inshallah")


# poisson_model.load_state_dict(torch.load("Poisson_statedict" + img_data_key + ".inshallah"))
# poisson_model.eval();

In [21]:
output

{'val_correlation': array([-0.00216921,  0.06089262,  0.07287578,  0.09017557,  0.11448877,
         0.12299193,  0.14777163,  0.17114505,  0.19159639,  0.21444872,
         0.226796  ,  0.2436114 ,  0.25637776,  0.2629218 ,  0.27408484,
         0.2732739 ,  0.28695077,  0.2864941 ,  0.2904621 ,  0.29799125,
         0.2989345 ,  0.29993096,  0.30322784,  0.306771  ,  0.30526882,
         0.30340692,  0.30748805,  0.30622983,  0.3083354 ,  0.31233382,
         0.3139222 ,  0.31518897,  0.31230223,  0.31495655,  0.318402  ,
         0.31611818,  0.31726065,  0.3171519 ,  0.31794217,  0.318402  ,
         0.31376648,  0.3244521 ,  0.31940612,  0.32657978,  0.3215325 ,
         0.32570863,  0.3239352 ,  0.32291067,  0.32657978,  0.32356533,
         0.32605797,  0.32780504,  0.3246255 ,  0.32496527,  0.32512647,
         0.32743704], dtype=float32),
 'val_loss': array([4523065. , 2475356.2, 2464938.5, 2454380.5, 2422110.5, 2448570.2,
        2380002.5, 2359384.2, 2326209.5, 2291545.5, 22

In [22]:
score

0.32780504

val_correlation 0.32643822

val_loss 2107371.0

train_loss 17724226.0

---

# Compare DEIs/MEIs

### DEI data

In [ ]:
idx = np.array([(dat["animal_id"] == imagenet_key["animal_id"]) & (dat["session"] != imagenet_key["session"]) & (dat["scan_idx"] != imagenet_key["scan_idx"]) for dat in datasets])
dei_key = np.array(datasets)[idx].item()

assert dei_key["scan_purpose"] == "dei_control_pair"
paths = ["./data/static{}-{}-{}-GrayImageNetDEIInfo-7bed7f7379d99271be5d144e5e59a8e7.zip".format(dei_key["animal_id"], dei_key["session"], dei_key["scan_idx"])]
dei_data_key = extract_data_key(paths[0])

dataset_config = {'paths': paths, 
                  'batch_size': 64, 
                  'seed': random_seed,
                  'return_test_sampler': True,
                  'tier': "test",
                  'loader_outputs': ["images", 'responses', 'trial_idx', "dei_unit_ids", "dei_src_unit_ids", "dei_mean_distances"],
                  'normalize': True,
                  'exclude': ["images", "trial_idx", "dei_unit_ids", "dei_src_unit_ids", "dei_mean_distances"],
                  'subtract_behavior_mean': True}

dei_dataloaders = static_loaders(**dataset_config)

dei_dataset = dei_dataloaders["test"][dei_data_key].dataset

In [ ]:
images, responses, trial_idxs, dei_unit_ids, dei_src_unit_ids, dei_mean_distances = [], [], [], [], [], []
for image, response, trial_idx, dei_unit_id, dei_src_unit_id, dei_mean_distance in dei_dataloaders["test"][dei_data_key]:
    if (len(response) == 20) & (torch.unique(dei_mean_distance <= 10)):
        images.append(image)
        responses.append(response)
        trial_idxs.append(trial_idx)
        dei_unit_ids.append(dei_unit_id)
        dei_src_unit_ids.append(dei_src_unit_id)
        dei_mean_distances.append(dei_mean_distance)
images = torch.stack(images)
responses = torch.stack(responses)
trial_idxs = torch.stack(trial_idxs).cpu().data.numpy()
dei_unit_ids = torch.stack(dei_unit_ids).cpu().data.numpy()
dei_src_unit_ids = torch.stack(dei_src_unit_ids).cpu().data.numpy()
dei_mean_distances = torch.stack(dei_mean_distances).cpu().data.numpy()

In [ ]:
# Get possible unit ids (in the source-dataset frame)
possible_src_unit_ids = np.unique(dei_src_unit_ids, axis=1).squeeze()

# Sort according to mean distances (increasing)
src_sort_idx = np.argsort(np.unique(dei_mean_distances, axis=1).squeeze())
possible_src_unit_ids = possible_src_unit_ids[src_sort_idx]

# Remove duplicates (from several DEIs/MEI)
_, idx = np.unique(possible_src_unit_ids, return_index=True)
possible_src_unit_ids = possible_src_unit_ids[np.sort(idx)]

In [ ]:
zig_model = Ensemble(zig_se2d_fullgaussian2d, zig_model_config, img_dataloaders, "ZIG_statedict" + img_data_key, np.arange(5), device=device)
poisson_model = Ensemble(poisson_se2d_fullgaussian2d, poisson_model_config, img_dataloaders, "Poisson_statedict" + img_data_key, np.arange(5), device=device)

# zig_model.shifter = None
# poisson_model.shifter = None
# poisson_model.modulator = None

In [ ]:
zig_means = np.full((3, len(possible_src_unit_ids)), np.nan)
zig_variances = np.full((3, len(possible_src_unit_ids)), np.nan)
poisson_means = np.full((3, len(possible_src_unit_ids)), np.nan)
poisson_variances = np.full((3, len(possible_src_unit_ids)), np.nan)
real_resp_means = np.full((3, len(possible_src_unit_ids)), np.nan)
real_resp_vars = np.full((3, len(possible_src_unit_ids)), np.nan)
imgs = np.full((3, len(possible_src_unit_ids), 1, 36, 64), np.nan)
for i, possible_src_unit_id in enumerate(possible_src_unit_ids):
    image_idx = np.unique(np.where(dei_src_unit_ids == possible_src_unit_id)[0])

    dei_neuron_id = np.unique(dei_unit_ids[image_idx]).item()
    src_neuron_id = np.unique(dei_src_unit_ids[image_idx]).item()
    src_neuron_idx = np.where(img_dataset.neurons.unit_ids == src_neuron_id)[0].item()
    dei_neuron_idx = np.where(dei_dataset.neurons.unit_ids == dei_neuron_id)[0].item()

    img = torch.unique(images[image_idx], dim=1).squeeze(1)
    mei_dei_idx = [0, 1, 2]
    if len(image_idx) != 3:
        idx = np.where(np.isin(dei_dataset.trial_info.trial_idx, trial_idxs[image_idx]))[0]
        if 'mask_fixed_mei' in np.unique(dei_dataset.trial_info.frame_image_class[idx]):
            mei_dei_idx = [0, 1] if len(image_idx) == 2 else [0]
        else:
            mei_dei_idx = [1, 2] if len(image_idx) == 2 else [1]
    imgs[mei_dei_idx, i, :, :, :] = img.cpu().data
    
    # TODO: Keep this line?
#     img = torch.stack([((im - im.mean()) / (im.std())) for im in img.squeeze()])[:, None]

    behavior = torch.zeros((img.shape[0], 3)).to(device)
    pupil_center = torch.zeros((img.shape[0], 2)).to(device)
    
    zig_means_ = zig_model.predict_mean(img, data_key=img_data_key, behavior=behavior, pupil_center=pupil_center).cpu().data.numpy()
    zig_variances_ = zig_model.predict_variance(img, data_key=img_data_key, behavior=behavior, pupil_center=pupil_center).cpu().data.numpy()
    poisson_means_ = poisson_model.predict_mean(img, data_key=img_data_key, behavior=behavior, pupil_center=pupil_center).cpu().data.numpy()
    poisson_variances_ = poisson_model.predict_variance(img, data_key=img_data_key, behavior=behavior, pupil_center=pupil_center).cpu().data.numpy()

    zig_means[mei_dei_idx, i] = zig_means_[:, src_neuron_idx]
    zig_variances[mei_dei_idx, i] = zig_variances_[:, src_neuron_idx]
    poisson_means[mei_dei_idx, i] = poisson_means_[:, src_neuron_idx]
    poisson_variances[mei_dei_idx, i] = poisson_variances_[:, src_neuron_idx]
    
    real_resp_means[mei_dei_idx, i] = np.mean(responses[image_idx].cpu().data.numpy(), axis=1)[:, dei_neuron_idx]
    real_resp_vars[mei_dei_idx, i] = np.var(responses[image_idx].cpu().data.numpy(), axis=1)[:, dei_neuron_idx]
keep_idx = ~np.isnan(zig_means).any(axis=0)

### Compare Zhiwei Model with Konstantin model

In [ ]:
with open(r"group233_mei_dei_resps.pkl", "rb") as input_file:
    e = pickle.load(input_file).T

In [ ]:
fig, axes = plt.subplots(4, 1, figsize=(15, 10), dpi=150, sharex=True, sharey=True)
fontsize = 10


y_zhiwei = e / e[0, :]
y_zig = zig_means / zig_means[0, :]
y_zig = y_zig[:, keep_idx]
y_poisson = poisson_means / poisson_means[0, :]
y_poisson = y_poisson[:, keep_idx]
y_real = real_resp_means / real_resp_means[0, :]
y_real = y_real[:, keep_idx]
x = np.arange(y_real.shape[1])

# Zhiwei
for i in range(3):
    axes[0].plot(x, y_zhiwei[i,:], ls="", marker="x")

# ZIG
for i in range(3):
    axes[1].plot(x, y_zig[i,:], ls="", marker="x")
    
# Poisson
for i in range(3):
    axes[2].plot(x, y_poisson[i,:], ls="", marker="x")
    axes[2].ticklabel_format(useOffset=False)
    
# Real data
for i, label in enumerate(["MEI", "DEI1", "DEI2"]):
    axes[3].plot(x, y_real[i,:], ls="", marker="x", label=label)
    
    
axes[0].set_title("Zhiwei model", fontsize=fontsize*1.3)
axes[1].set_title("ZIG model", fontsize=fontsize*1.3)
axes[2].set_title("Poisson model", fontsize=fontsize*1.3)
axes[3].set_title("Real data  (averaged over 20 repeats)", fontsize=fontsize*1.3)
axes[3].set_xlabel("neurons", fontsize=fontsize*1.2)
axes[3].set_ylabel(r"$\frac{resp}{resp(MEI)}$", fontsize=fontsize*1.2)

axes[0].set(ylim=[0, 1.5])

# axes[3].legend(bbox_to_anchor=(0.15, 1., 0, 0), frameon=True, fontsize=fontsize*.8)
axes[3].legend(bbox_to_anchor=(0.1, 1.1, 0, 0), frameon=True, fontsize=fontsize*.8)

sns.despine(trim=True)
# fig.savefig("Zhiwei_Model_Comparison" + ".png", bbox_inches="tight", transparent=False)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_zhiwei[0], y_poisson[0])
ax.scatter(y_zhiwei[0], y_zig[0])
ax.scatter(y_zhiwei[0], y_real[0])
ax.plot([ax.get_xlim()[0], ax.get_xlim()[1]], [ax.get_ylim()[0], ax.get_ylim()[1]])

#### Rank correlation

In [ ]:
zig_zhiwei, poisson_zhiwei, zig_real, poisson_real, zhiwei_real = [], [], [], [], []
for i in range(e.shape[-1]):
    zig_zhiwei_, p = spearmanr(e[:, i], zig_means[:, keep_idx][:, i], axis=0)
    poisson_zhiwei_, p = spearmanr(e[:, i], poisson_means[:, keep_idx][:, i], axis=0)
    zig_real_, p = spearmanr(real_resp_means[:, keep_idx][:, i], zig_means[:, keep_idx][:, i], axis=0)
    poisson_real_, p = spearmanr(real_resp_means[:, keep_idx][:, i], poisson_means[:, keep_idx][:, i], axis=0)
    zhiwei_real_, p = spearmanr(real_resp_means[:, keep_idx][:, i], e[:, i], axis=0)
    
    zig_zhiwei.append(zig_zhiwei_)
    poisson_zhiwei.append(poisson_zhiwei_)
    zig_real.append(zig_real_)
    poisson_real.append(poisson_real_)
    zhiwei_real.append(zhiwei_real_)

print("ZIG     |  Zhiwei: {:.5f}".format(np.mean(zig_zhiwei)))
print("Poisson |  Zhiwei: {:.5f}".format(np.mean(poisson_zhiwei)))
print("ZIG     |  Real:   {:.5f}".format(np.mean(zig_real)))
print("Poisson |  Real:   {:.5f}".format(np.mean(poisson_real)))
print("Zhiwei  |  Real:   {:.5f}".format(np.mean(zhiwei_real)))

#### Full Correlation

In [ ]:
zig_zhiwei = np.mean(corr(e, zig_means[:, keep_idx], axis=0))
poisson_zhiwei = np.mean(corr(e, poisson_means[:, keep_idx], axis=0))
zig_real = np.mean(corr(real_resp_means[:, keep_idx], zig_means[:, keep_idx], axis=0))
poisson_real = np.mean(corr(real_resp_means[:, keep_idx], poisson_means[:, keep_idx], axis=0))
zhiwei_real = np.mean(corr(real_resp_means[:, keep_idx], e, axis=0))

print("ZIG     |  Zhiwei: {:.5f}".format(np.mean(zig_zhiwei)))
print("Poisson |  Zhiwei: {:.5f}".format(np.mean(poisson_zhiwei)))
print("ZIG     |  Real:   {:.5f}".format(np.mean(zig_real)))
print("Poisson |  Real:   {:.5f}".format(np.mean(poisson_real)))
print("Zhiwei  |  Real:   {:.5f}".format(np.mean(zhiwei_real)))

___

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(9, 3), dpi=150)
color = "navy"
ec="white"
fontsize = 15

axes[0].scatter(real_resp_means.flatten(), real_resp_vars.flatten(), color=color, ec=ec)
axes[0].set_xlabel("Means", fontsize=fontsize)
axes[0].set_ylabel("Variances", fontsize=fontsize)
axes[0].set_title("Real Data", fontsize=fontsize*1.3)

axes[1].scatter(zig_means.flatten(), zig_variances.flatten(), color=color, ec=ec)
axes[1].set_title("ZIG", fontsize=fontsize*1.3)
axes[1].plot([axes[1].get_xlim()[0], axes[1].get_xlim()[1]], [axes[1].get_xlim()[0], axes[1].get_xlim()[1]], ls="--", color="grey", label="Poisson")
# axes[1].set(xlim=[0, 15], ylim=[0, 400])
axes[1].legend(frameon=False, bbox_to_anchor=[.4,.8,0,0])
sns.despine(trim=True)

# fig.savefig("mean_variance_comparison" + ".png", bbox_inches="tight", transparent=False)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(9, 3), dpi=150)
color = "navy"
ec="white"
fontsize = 15

axes[0].plot(real_resp_means, real_resp_vars, marker="", lw=1)
axes[0].set_xlabel("Means", fontsize=fontsize)
axes[0].set_ylabel("Variances", fontsize=fontsize)
axes[0].set_title("Real Data", fontsize=fontsize*1.3)

axes[1].plot(zig_means, zig_variances, marker="", lw=1)
axes[1].set_title("ZIG", fontsize=fontsize*1.3)
axes[1].plot([axes[1].get_xlim()[0], axes[1].get_xlim()[1]], [axes[1].get_xlim()[0], axes[1].get_xlim()[1]], ls="--", color="grey", label="Poisson")
# axes[1].set(xlim=[0, 15], ylim=[0, 400])
axes[1].legend(frameon=False, bbox_to_anchor=[.4,.8,0,0])
sns.despine(trim=True)

# fig.savefig("mean_variance_comparison" + ".png", bbox_inches="tight", transparent=False)